In [ ]:
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import dill as pickle
import tensorflow as tf
import matplotlib.pyplot as plt
# mpl_style = "/home/zanardi/Workspace/Research/styles/matplotlib/paper_1column.mplstyle",
# plt.style.use(mpl_style)

In [ ]:
sys.path.append("/home/zanardi/Codes/ML/ROMAr/romar/")
from romar import env
from romar import const
from romar import utils
from romar import backend as bkd
from romar.systems import BoxAd
from romar.roms import PCA
from romar import backend as bkd

import romar.postproc.plotting as pltt

In [ ]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

In [ ]:
path_to_dtb = "/home/zanardi/Codes/ML/ROMAr/romar/examples/database"
path_to_basis1 = "/home/zanardi/Codes/ML/ROMAr/run/test1/max_mom_2_Tfull/cobras_basis.p"
path_to_basis2 = "/home/zanardi/Codes/ML/ROMAr/run_old2/rad_on_test10/max_mom_2a/cobras_basis.p"
# path_to_basis = "/home/zanardi/Codes/ML/ROMAr/run/test1/max_mom_2_Tfull/cobras_basis.p"
path_to_mask = "/home/zanardi/Codes/ML/ROMAr/run/rad_on_test1/max_mom_2/rom_mask.txt"
path_to_data = "/home/zanardi/Codes/ML/ROMAr/runs/run01/data/test/"
# cases = ["lrho_lT", "lrho_hT", "hrho_lT", "hrho_hT", "mrho_mT"]
cases = ["mrho_mT"]

In [ ]:
system = BoxAd(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  rad_dtb=path_to_dtb + "/rates/rad_fit.p",
  use_rad=True,
  use_proj=False,
  use_tables=False
)

In [ ]:
# basis1 = pickle.load(open(path_to_basis1, "rb"))
# basis1["xref"], basis1["xscale"]

In [ ]:
basis1 = pickle.load(open(path_to_basis1, "rb"))
basis2 = pickle.load(open(path_to_basis2, "rb"))

In [ ]:
# for k in ("xref", "xscale"):
#   err = np.abs((basis1[k] - basis[k]) / basis1[k])
#   print(err.max())
# for k in ("phi", "psi"):
#   err = np.abs((basis1[k][34] - basis[k][34]) / basis1[k][34])
#   print(err.max())

In [ ]:
ph2 = basis2["phi"][34][:,:10]
ps2 = basis2["psi"][34][:,:10]
P2 = ph2 @ np.linalg.inv(ps2.T @ ph2) @ ps2.T

In [ ]:
ph1 = basis1["phi"][34][:,:10]
ps1 = basis1["psi"][34][:,:10]
P1 = ph1 @ np.linalg.inv(ps1.T @ ph1) @ ps1.T

In [ ]:
np.linalg.norm(P1 - P2)/np.linalg.norm(P1)

In [ ]:
cov_mats_path = "/home/zanardi/Codes/ML/ROMAr/runs/run01/max_mom_2/cobras_cov_mats.p"
X, Y = pickle.load(open(cov_mats_path, "rb"))

In [ ]:
Y.shape

In [ ]:
np.linalg.norm(X[1:-2], axis=0).max()

In [ ]:
norm = np.linalg.norm(Y[:-2], axis=0)
i = np.where(norm > 1e3)[0]
i

In [ ]:
!pip install seaborn

In [ ]:
import numpy as np; np.random.seed(1)
import matplotlib.pyplot as plt
import seaborn as sns

n = norm[i]
sns.distplot(n, bins=np.arange(0,16), hist_kws=dict(ec="k"))

plt.show()

In [ ]:
for i in range(10):
  nb = str(i+1)
  b = basis["phi"][30][:,i]
  pltt.plot_dist_2d(
    x=np.arange(len(b)),
    y=b,
    labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % ("phi", nb)],
    scales=["linear", "linear"],
    markersize=1,
    # figname=path + f"/{name}_{nb.zfill(2)}",
    save=False,
    show=True
  )

In [ ]:
for i in range(10):
  nb = str(i+1)
  b = basis["psi"][30][:,i]
  pltt.plot_dist_2d(
    x=np.arange(len(b)),
    y=b,
    labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % ("psi", nb)],
    scales=["linear", "linear"],
    markersize=1,
    # figname=path + f"/{name}_{nb.zfill(2)}",
    save=False,
    show=True
  )

Load test cases

In [ ]:
testcases = {k: utils.load_case(filename=path_to_data + "case_" + k + ".p") for k in cases}

In [ ]:
for k, testcase in testcases.items():
  # Testcase
  print(k)
  print("="*20)
  t, y, rho = [testcase[k] for k in ("t", "y", "rho")]
  system.mix.set_rho(rho)
  n, Th, Te = system.get_prim(y)
  # Species
  for s in system.mix.species.values():
    print(s.name)
    for i in s.indices:
      plt.loglog(t, n[i])
    plt.show()
    plt.close()
  # Temperatures
  print("Th-Te")
  plt.semilogx(t, Th)
  plt.semilogx(t, Te)
  plt.show()
  plt.close()

Test ROM

In [ ]:
rdim = 4

system.rom.build(
  phi=basis["phi"][rdim],
  psi=basis["psi"][rdim],
  **{k: basis[k] for k in ("mask", "xref", "xscale")}
)

In [ ]:
labels = {
  "Th": "$T_h$",
  "Te": "$T_e$",
  "Ar": "\\text{Ar}",
  "Arp": "\\text{Ar$^+$}",
  "em": "\\text{e$^-$}"
}

In [ ]:
isol, _ = system.compute_sol_rom(
  filename=path_to_data + "case_" + cases[-1] + ".p"
)

In [ ]:
tf.nest.map_structure(np.shape, isol)

In [ ]:
t = isol.pop("t")
err = {"CoBRAS": isol.pop("err")}
err["POD"] = err["CoBRAS"]
sol = isol
sol["CoBRAS"] = sol.pop("ROM")
sol["POD"] = sol["CoBRAS"]

In [ ]:
tf.nest.map_structure(np.shape, sol)

In [ ]:
tf.nest.map_structure(np.shape, err)

In [ ]:
for k, testcase in testcases.items():
  # Testcase
  print(k)
  print("="*20)
  # FOM solution
  t, y0, y, rho = [testcase[k] for k in ("t", "y0", "y", "rho")]
  system.mix.set_rho(rho)
  prim_fom = system.get_prim(y)
  mom_fom = system.compute_mom(prim_fom[0])
  # ROM solution
  yrom, _ = system.solve_rom(t, y0, rho)
  prim_rom = system.get_prim(yrom)
  mom_rom = system.compute_mom(prim_rom[0])
  nt = len(yrom.T)
  nt
  # Temperatures
  print("Th-Te")
  for i in range(2):
    plt.semilogx(t[:nt], prim_fom[1+i][:nt])
    plt.semilogx(t[:nt], prim_rom[1+i][:nt], ls="--", color=plt.gca().lines[-1].get_color(), lw=2.0)
  plt.show()
  plt.close()
  # Species
  for s in system.mix.species.values():
    print(s.name)
    for i in s.indices:
      plt.loglog(t[:nt], prim_fom[0][i,:nt])
      plt.loglog(t[:nt], prim_rom[0][i,:nt], ls="--", color=plt.gca().lines[-1].get_color(), lw=2.0)
    plt.show()
    plt.close()
  # Moments
  for s in system.mix.species.values():
    if s.name != "em":
      for m in range(2):
        print(s.name+f" - Moment: {m}")
        plt.semilogx(t[:nt], mom_fom[s.name][f"m{m}"][:nt])
        plt.semilogx(t[:nt], mom_rom[s.name][f"m{m}"][:nt], ls="--", color=plt.gca().lines[-1].get_color(), lw=2.0)
        plt.yscale("log" if m == 0 else "linear")
        plt.show()
        plt.close()